In [ ]:
import os
import re
import requests
import pandas as pd
from urllib.parse import urlparse


In [2]:
!pip install openpyxl


In [ ]:


# ================= CONFIG =================

OUTPUT_COLLEGE_INFO_FOLDER = "output_college_info_6_7"
PDF_DOWNLOADS_FOLDER = "pdf_downloads_final"
DOWNLOAD_TIMEOUT = 90

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept": "application/pdf,application/octet-stream,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
}

# ================= HELPERS =================

def normalize_pdf_url(url: str) -> str:
    """
    NO-OP by design.
    URL is returned exactly as provided in Excel.
    """
    return url

YEAR_PATTERN = re.compile(
    r"(19\d{2}|20\d{2})\s*(?:[-_/]\s*(\d{2}|19\d{2}|20\d{2}))?",
    re.IGNORECASE
)


def split_links_with_text(cell_value: str):
    """
    Split a cell into logical (url + text) chunks.
    A chunk starts at http(s):// and ends right before the next http(s)://
    """
    if not isinstance(cell_value, str):
        return []

    text = cell_value.replace("\r", " ").replace("\n", " ").strip()

    parts = re.split(r"(https://|http://)", text)
    chunks = []

    i = 1
    #print(f"Original text: {text}...")
    while i < len(parts) - 1:
        combined = (parts[i] + parts[i + 1]).strip()
        chunks.append(combined)
        #print(f"[DEBUG] Chunk: {combined}...")
        i += 2
    #print(f"\n")
    return chunks


def extract_pdf_link_from_unit(unit: str):
    """
    Extract ONE PDF URL from a unit.
    - Must start with http:// or https://
    - Must end with .pdf
    - Allows breaks (space, newline, hyphen)
    Returns the first valid PDF or None.
    """
    if not isinstance(unit, str):
        return None

    raw = unit.replace("\r", "")

    pattern = re.compile(
        r"(https?://(?:[^\s]|[\s\-]){1,1000}?\.pdf)",
        re.IGNORECASE
    )

    match = pattern.search(raw)
    if not match:
        return None

    # cleaned = match.group(1)
    # cleaned = cleaned.replace("\n", "").replace(" ", "")
    # cleaned = cleaned.replace("\t", "").replace("\u00a0", "")
    # cleaned = cleaned.replace("-/", "/")

    return match.group(1)


CATEGORY_KEYWORDS = {
    "mandatory_disclosure": ["mandatory", "mandatory disclosure", "mandatory_disclosure", "mandatory disclosures", "mandatory_disclosures", "statutory disclosure", "statutory_disclosure", "statutory disclosures", "statutory_disclosures", "aicte mandatory disclosure", "aicte_mandatory_disclosure", "aicte disclosure", "aicte_disclosure", "mandatory discloser", "mandatory_discloser", "mandatory discloure", "mandatory_discloure", "disclosure"],

    "naac": ["naac", "naac accreditation", "naac_accreditation", "assessment & accreditation", "assessment and accreditation", "assessment_accreditation", "assessment_and_accreditation", "ssr", "dvv", "nacc", "qim", "qnm"],

    "nba": ["nba", "nba accreditation", "nba_accreditation", "national board of accreditation", "national_board_of_accreditation"],

    "nirf": ["nirf", "nirf ranking", "nirf_ranking", "nirf india ranking", "nirf_india_ranking", "national institutional ranking framework", "national_institutional_ranking_framework"],

    "iqac": ["iqac", "internal quality assurance cell", "internal_quality_assurance_cell", "quality assurance cell", "quality_assurance_cell", "quality assurance committee", "quality_assurance_committee"],

    "aicte": ["aicte", "all india council for technical education", "all_india_council_for_technical_education", "aicte approval", "aicte_approval", "aicte extension of approval", "aicte_extension_of_approval"],

    "aqar": ["aqar", "annual quality assurance report", "annual_quality_assurance_report"],

    "ariia": ["ariia", "atal", "atal ranking", "atal_ranking", "atal ranking of institutions on innovation achievements", "atal_ranking_of_institutions_on_innovation_achievements"],

    "accreditation": ["accreditation", "accredited", "accreditations", "download", "affiliation","placement"],

    "criteria": ["crit", "criteria", "criterion", "criteria1", "criteria2", "criteria3", "criteria4", "criteria5", "criteria6", "criteria7"],

    "criteria_1": ["criteria 1", "criteria-1", "criteria_1", "criterion 1", "criterion-1", "criterion_1", "criterion1", "criteria1"],

    "criteria_2": ["criteria 2", "criteria-2", "criteria_2", "criterion 2", "criterion-2", "criterion_2", "criterion2", "criteria2"],

    "criteria_3": ["criteria 3", "criteria-3", "criteria_3", "criterion 3", "criterion-3", "criterion_3", "criterion3", "criteria3"],

    "criteria_4": ["criteria 4", "criteria-4", "criteria_4", "criterion 4", "criterion-4", "criterion_4", "criterion4", "criteria4"],

    "criteria_5": ["criteria 5", "criteria-5", "criteria_5", "criterion 5", "criterion-5", "criterion_5", "criterion5", "criteria5"],

    "criteria_6": ["criteria 6", "criteria-6", "criteria_6", "criterion 6", "criterion-6", "criterion_6", "criterion6", "criteria6"],

    "criteria_7": ["criteria 7", "criteria-7", "criteria_7", "criterion 7", "criterion-7", "criterion_7", "criterion7", "criteria7"],
}

def normalize_text_for_match(text: str) -> str:
    """
    Normalize text by:
    - lowercasing
    - replacing non-alphanumeric characters with '_'
    - collapsing multiple '_' into single '_'
    - stripping leading/trailing '_'
    """
    if not isinstance(text, str):
        return ""

    text = text.lower()
    text = re.sub(r"[^a-z0-9]", "_", text)
    text = re.sub(r"_+", "_", text)
    return text.strip("_")


import unicodedata

def normalize_for_year_detection(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # Unicode normalization (fixes – — etc.)
    text = unicodedata.normalize("NFKD", text)

    # Replace common separators with space
    text = re.sub(r"[._/\\\-–—]+", " ", text)

    # Remove URL encoding artifacts
    text = text.replace("%20", " ")

    # Collapse whitespace
    text = re.sub(r"\s+", " ", text)

    return text.strip()

def extract_document_years(text: str) -> list[int]:
    if not isinstance(text, str):
        return []

    text = normalize_for_year_detection(text)

    years = set()

    # 1️⃣ Capture standalone years (not embedded in numbers)
    for y in re.findall(r"(?<!\d)(19\d{2}|20\d{2})(?!\d)", text):
        years.add(int(y))

    # 2️⃣ Capture year ranges like 2027-28, 2027/28, 2027–28, 2027_28
    for y1, y2 in re.findall(
        r"(?<!\d)(19\d{2}|20\d{2})\s*[-/–—_]\s*(\d{2}|19\d{2}|20\d{2})(?!\d)",
        text
    ):
        y1 = int(y1)

        if len(y2) == 2:
            # 2027-28 → 2028
            y2 = int(str(y1)[:2] + y2)
        else:
            y2 = int(y2)

        years.add(y1)
        years.add(y2)

    return sorted(years)


def unit_has_old_year(text: str) -> bool:
    years = extract_document_years(text)

    # Production-safe rule:
    # No year → discard
    if not years:
        return False

    # Keep ONLY if at least one year >= 2019
    return max(years) < 2019


def safe_filename_from_url(url: str, index: int ) -> str:
    index = 0 
    """
    Create filename from URL by:
    - removing scheme (http/https)
    - removing domain
    - keeping full path
    - making it filesystem-safe
    """
    parsed = urlparse(url)

    # Take full path without leading slash
    path = parsed.path.lstrip("/")

    # Fallback safety
    if not path.lower().endswith(".pdf"):
        path = "document.pdf"

    # Replace path separators and illegal chars
    filename = re.sub(r"[\\/]", "_", path)
    filename = re.sub(r"[^\w.\-]", "_", filename)

    return filename


def extract_years(text: str):
    if not isinstance(text, str):
        return []
    return [int(y) for y in re.findall(r"\b(19\d{2}|20\d{2})\b", text)]

def normalize_column_name(col: str) -> str:
    col = col.lower()
    if col.endswith("_links"):
        col = col[:-6]
    return col


def get_target_year_folder(unit_text: str, base_output_dir: str):
    years = extract_document_years(unit_text)

    if years:
        year_folder = str(max(years))
    else:
        year_folder = "no_year"

    folder_path = os.path.join(base_output_dir, year_folder)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

def nirf_with_innovation_or_management(unit_text: str) -> bool:
    """
    Returns True if:
    - 'nirf' is present
    AND
    - innovation OR management is present (any canonical / short form)
    """
    if not isinstance(unit_text, str):
        return False

    text = normalize_text_for_match(unit_text).lower()

    nirf_terms = {
        "nirf",
        "national_institutional_ranking_framework"
    }

    innovation_terms = {
        "innovation", 
        "innv",

    }

    management_terms = {
        "management",
        "mgmt", "mgmnt", "mgt",
        "mba", "pgdm",
        "business_administration"
    }

    has_nirf = any(term in text for term in nirf_terms)
    has_innovation = any(term in text for term in innovation_terms)
    has_management = any(term in text for term in management_terms)

    return has_nirf and (has_innovation or has_management)


ALL_CATEGORY_KEYWORDS = {
    normalize_text_for_match(kw.lower())
    for keywords in CATEGORY_KEYWORDS.values()
    for kw in keywords
}

def handle_depth_0(cell_value, output_dir, pdf_count):
    chunks = split_links_with_text(cell_value)
    if not chunks:
        return pdf_count

    keywords = ALL_CATEGORY_KEYWORDS

    matched_chunks = []

    for unit in chunks:
        normalized_unit = normalize_text_for_match(unit.lower())

        if any(kw in normalized_unit for kw in keywords):
            matched_chunks.append(unit)
        else:
            print(f"[SKIP][NO-KEYWORD-MATCH] {unit[:120]}...")


    if not matched_chunks:
        print(f"[SKIP][DEPTH0][NO-MATCH]")
        return pdf_count

    for unit in matched_chunks:
        # ✅ NEW: discard NIRF + Innovation / Management
        if nirf_with_innovation_or_management(unit):
            print("[SKIP][NIRF-INNOVATION/MANAGEMENT]", unit)
            continue

        link = extract_pdf_link_from_unit(unit)
        if not link:
            continue

        # ✅ UNIT-level year check
        if unit_has_old_year(unit) or unit_has_old_year(link):
            print(f"[SKIP][OLD-YEAR] {unit}")
            continue

        if link in seen_pdf_urls:
            print(f"[SKIP][DUPLICATE] {link}")
            continue

        seen_pdf_urls.add(link)
        pdf_count += 1
        filename = safe_filename_from_url(link, pdf_count)
        #file_path = os.path.join(output_dir, filename)

        # ✅ ADD THIS
        target_dir = get_target_year_folder(unit, output_dir)
        file_path = os.path.join(target_dir, filename)

        if os.path.exists(file_path):
            print(f"[SKIP] {filename}")
            continue

        try:
            print(f"[DOWNLOAD] {link}")
            with requests.get(
                link,
                headers=HEADERS,
                stream=True,
                timeout=DOWNLOAD_TIMEOUT
            ) as r:
                r.raise_for_status()
                with open(file_path, "wb") as f:
                    for chunk in r.iter_content(8192):
                        if chunk:
                            f.write(chunk)
        except Exception as e:
            print(f"[ERROR] {link} -> {e}")

    return pdf_count


def handle_depth_gt_0(cell_value, output_dir, pdf_count):
    chunks = split_links_with_text(cell_value)
    if not chunks:
        return pdf_count

    for unit in chunks:

        # ✅ NEW: discard NIRF + Innovation / Management
        if nirf_with_innovation_or_management(unit):
            print("[SKIP][NIRF-INNOVATION/MANAGEMENT]", unit)
            continue

        link = extract_pdf_link_from_unit(unit)
        if not link:
            continue

        # ✅ UNIT-level year check
        if unit_has_old_year(unit) or unit_has_old_year(link):
            print(f"[SKIP][OLD-YEAR] {unit}")
            continue

        if link in seen_pdf_urls:
            print(f"[SKIP][DUPLICATE] {link}")
            continue

        seen_pdf_urls.add(link)
        pdf_count += 1
        filename = safe_filename_from_url(link, pdf_count)
        #file_path = os.path.join(output_dir, filename)
        
        # ✅ ADD THIS
        target_dir = get_target_year_folder(unit, output_dir)
        file_path = os.path.join(target_dir, filename)

        if os.path.exists(file_path):
            print(f"[SKIP] {filename}")
            continue

        try:
            print(f"[DOWNLOAD] {link}")
            with requests.get(
                link,
                headers=HEADERS,
                stream=True,
                timeout=DOWNLOAD_TIMEOUT
            ) as r:
                r.raise_for_status()
                with open(file_path, "wb") as f:
                    for chunk in r.iter_content(8192):
                        if chunk:
                            f.write(chunk)
        except Exception as e:
            print(f"[ERROR] {link} -> {e}")

    return pdf_count



# ================= CORE =================

def download_pdfs_from_excel(excel_path):
    global seen_pdf_urls
    seen_pdf_urls = set()
    excel_name = os.path.basename(excel_path)
    excel_stem = os.path.splitext(excel_name)[0]

    output_dir = os.path.join(
        PDF_DOWNLOADS_FOLDER,
        f"pdf_download_{excel_stem}"
    )
    os.makedirs(output_dir, exist_ok=True)

    print(f"\n[INFO] Reading Excel: {excel_path}")
    print(f"[INFO] Output folder: {output_dir}")

    df = pd.read_excel(excel_path, engine="openpyxl")

    if "row_type" not in df.columns:
        print("[WARN] 'row_type' column missing — skipping")
        return

    pdf_count = 0

    for _, row in df.iterrows():
        if str(row.get("row_type", "")).strip().lower() != "pdf":
            continue

        depth = int(row.get("depth", -1))

        for col in df.columns:
            if col in {"row_type", "depth"}:
                continue

            cell_value = row[col]

            if depth == 0:
                #print(f"[PROCESS][DEPTH {depth}] Column: {col}")
                pdf_count = handle_depth_0(
                    cell_value,
                    output_dir,
                    pdf_count
                )
            else:
                #print(f"[PROCESS][DEPTH {depth}] Column: {col}")
                pdf_count = handle_depth_gt_0(
                    cell_value,
                    output_dir,
                    pdf_count
                )


    print(f"[DONE] PDFs attempted from {excel_name}: {pdf_count}")

# ================= RUN ALL EXCEL FILES =================

def run_for_all_excels():
    if not os.path.isdir(OUTPUT_COLLEGE_INFO_FOLDER):
        raise FileNotFoundError(OUTPUT_COLLEGE_INFO_FOLDER)

    excel_files = [
        f for f in os.listdir(OUTPUT_COLLEGE_INFO_FOLDER)
        if f.lower().endswith(".xlsx") and not f.startswith("~$")
    ]

    if not excel_files:
        print("[WARN] No Excel files found")
        return

    for excel in excel_files:
        excel_path = os.path.join(OUTPUT_COLLEGE_INFO_FOLDER, excel)
        download_pdfs_from_excel(excel_path)

    print("\n[ALL DONE] Processed all Excel files")



<>:61: SyntaxWarning: invalid escape sequence '\d'
<>:61: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_438802/2056021713.py:61: SyntaxWarning: invalid escape sequence '\d'
  '''


In [ ]:
#---------------------final function to run-------------------------------------
run_for_all_excels()

NameError: name 'run_for_all_excels' is not defined

In [ ]:
#download_pdfs_from_excel("output_college_info/college_info_96_Abacus_Institute_Of_Engineering_And_Management.xlsx")



[INFO] Reading Excel: output_college_info/college_info_pallavi_engineering_college.xlsx
[INFO] Output folder: pdf_downloads/pdf_download_college_info_pallavi_engineering_college
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/nirf/PEC NIRF-2025 Engineering 19691.pdf
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/nirf/PEC NIRF-2025 Innovation 19691.pdf
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/nirf/PEC NIRF-2025 Management 19691.pdf
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/nirf/PEC NIRF-2025 Overall 19691.pdf
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/assets/img/facilities/eoreport.pdf
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/admissions/PallaviBseats.pdf
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/assets/docs/HolidaysList.pdf
[DOWNLOAD] https://pallaviengineeringcollege.ac.in/mondatory-disclosure.pdf
[DOWNLOAD] https://www.pallaviengineeringcollege.ac.in/wp-content/uploads/2023/02/IITB_IITKGP_Workshops.pdf
[DOWNLOAD] https://www.pallav